In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Загружаем данные
file_path = "../npz/BTCUSD_1T.npz"
data = np.load(file_path, allow_pickle=True)
if 'data' in data:
    raw_data = data['data']
else:
    raise ValueError("Файл не содержит ключа 'data'")
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(raw_data, columns=columns)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
# При необходимости можно ограничить число точек:
#df = df.iloc[:50* 1000]


In [ ]:
########################################################################
# Вспомогательная функция для перевода (x, y, z) в индексы сетки с использованием объекта grid_size
def to_grid_3d(xx, yy, zz, bounds, grid_size):
    """
    Преобразует координаты (xx, yy, zz) в индексы сетки.
    
    :param xx, yy, zz: координаты точки.
    :param bounds: кортеж (x_min, x_max, y_min, y_max, z_min, z_max).
    :param grid_size: словарь с ключами 'x', 'y', 'z' для размеров сетки.
    :return: кортеж (i_x, i_y, i_z) – индексы точки в сетке.
    """
    (x_min, x_max, y_min, y_max, z_min, z_max) = bounds
    i_x = int((xx - x_min) / (x_max - x_min) * (grid_size['x'] - 1))
    i_y = int((yy - y_min) / (y_max - y_min) * (grid_size['y'] - 1))
    i_z = int((zz - z_min) / (z_max - z_min) * (grid_size['z'] - 1))
    # return (x_min + i_x * (grid_size['x'] - 1) / (x_max - x_min), i_y, i_z)
    return (i_x, i_y, i_z)

########################################################################
# Функция построения 3D-кэша фазового пространства с отдельными размерами сетки (через объект grid_size)
def build_phase_space_grid_3d(df, window_short, window_long, tau, grid_size):
    """
    Разбивает фазовое пространство (x, y, z) на 3D-сетку и заполняет кэш переходов.
    Здесь:
      x = price_diff (0-я производная сглаженного относительного изменения цены)
      y = первая производная (относительное изменение скорости)
      z = вторая производная (ускорение изменения относительного изменения)
    
    :param df: DataFrame с ценами.
    :param window_short: короткое окно сглаживания.
    :param window_long: длинное окно сглаживания.
    :param tau: шаг прогноза (в баров).
    :param grid_size: словарь с размерами сетки, например {'x':50, 'y':50, 'z':50}.
    :return: (cache, bounds), где
             cache = { (i_x, i_y, i_z) -> { future_x: count } }
             bounds = (x_min, x_max, y_min, y_max, z_min, z_max)
    """
    polyorder = 3

    # Сглаживаем цену
    smoothed_short = savgol_filter(df['close'].values, window_short, polyorder)
    smoothed_long  = savgol_filter(df['close'].values, window_long, polyorder)
    
    # Относительное изменение (в процентах)
    rel_price_diff = (smoothed_short - smoothed_long) / smoothed_long

    # Вычисляем три координаты: 
    # x = базовый сигнал (разность),
    # y = первая производная,
    # z = вторая производная.
    x = savgol_filter(rel_price_diff, window_short, polyorder, deriv=0)
    y = savgol_filter(rel_price_diff, window_short, polyorder, deriv=1)
    z = savgol_filter(rel_price_diff, window_short, polyorder, deriv=2)
    
    # Границы фазового пространства
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)
    z_min, z_max = np.min(z), np.max(z)
    bounds = (x_min, x_max, y_min, y_max, z_min, z_max)
    
    cache = {}  # { (i_x, i_y, i_z) -> { future_x: count } }
    n = len(x) - tau
    for t in range(n):
        cell_now = to_grid_3d(x[t], y[t], z[t], bounds, grid_size)
        cell_future = to_grid_3d(x[t+tau], y[t+tau], z[t+tau], bounds, grid_size)
        future_x = cell_future[0]  # Используем только x-компоненту будущей ячейки
        if cell_now not in cache:
            cache[cell_now] = {}
        if future_x not in cache[cell_now]:
            cache[cell_now][future_x] = 0
        cache[cell_now][future_x] += 1
    return cache, bounds

########################################################################
# Функция визуализации прогноза с использованием 3D-кэша (новый формат кеша)
def plot_price_forecast_with_heatmap_3d(df, caches, bounds, windows, tau, start_idx, end_idx, grid_size):
    """
    Строит график цен и наносит прогноз в виде точек,
    где цвет отражает частоту перехода. Для обратного прогнозирования используется только x-компонента из 3D-кэша.
    
    :param df: DataFrame с ценами (тестовая выборка).
    :param caches: [{}] transitions in phase space.
    :param bounds: [(x_min, x_max, y_min, y_max, z_min, z_max)] – min,max values for the phase spaces.
    :param windows: [(window_short, window_long)] window sizes that were used to build phase space.
    :param tau: forecase distance.
    :param start_idx: начало прогноза.
    :param end_idx: конец прогноза.
    :param grid_size: словарь с размерами сетки, например {'x':200, 'y':100, 'z':10}.
    """
    polyorder = 3
    (x_min, x_max, y_min, y_max, z_min, z_max) = bounds

    rel_price_diffs = []
    xs = []
    ys = []
    zs = []
    for i in range(len(caches)):
        window_short, window_long = windows[i]
        smoothed_short = savgol_filter(df['close'].values, window_short, polyorder)
        smoothed_long  = savgol_filter(df['close'].values, window_long, polyorder)
        rel_price_diff = (smoothed_short - smoothed_long) / smoothed_long
        rel_price_diffs.append(rel_price_diff)
        # Вычисляем x, y, z
        x = savgol_filter(rel_price_diff[i], window_short, polyorder, deriv=0)
        y = savgol_filter(rel_price_diff[i], window_short, polyorder, deriv=1)
        z = savgol_filter(rel_price_diff[i], window_short, polyorder, deriv=2)
        xs.append(x)
        ys.append(y)
        zs.append(z)

    # Используем ту же функцию to_grid_3d для преобразования
    def to_grid(xx, yy, zz):
        return to_grid_3d(xx, yy, zz, bounds, grid_size)

    future_points = []
    future_counts = []
    N = len(x)
    # тут дальше начинается сложное место, которое я пока не осилил написать.
    # надо для каждого момента времени t взять прогнозы от разных маштабов (кешей) и объединить их.
    # для этого можно пойти разными путями. 
    # 1. запускаем цикл по прогнозам первого маштаба (итерируемся по j), вычисляем future_price_diff_j, 
    #    заходим во вложенный цикл по прогнозам второго масштаба (итерируемся по k), вычисляем там future_price_diff_k,
    #    заходим в еще один вложенный цикл по прогнозам третьего масштаба (итерируемся по l), вычисляем там future_price_diff_l,
    #    и так далее. потом суммируем все future_price_diff_j, future_price_diff_k, future_price_diff_l, получаем 
    #    результирующий прогноз. кладем его в сетку и сохраяем в future_points, а количество в future_counts. но 
    #    вместо каунтов надо уже брать вероятности. для этого надо в каждом масштабе посчитать сумму всех каунтов и
    #    потом делить каждый каунт на эту сумму. однако, тут не понятно как сделать вложенные циклы, так как
    #    количество масштабов неизвестно заранее. поэтому надо как-то рекурсивно это делать.
    # 2. можно сразу сделать рекурсивную функцию, которая будет принимать на вход два прогноза и объединять их в один.
    #    потом применять эту функцию к каждой паре прогнозов. но тут тоже есть проблема - как сохранять объедененные прогнозы.
    #    если просто брать сдвиг цены одного маштбара, прибавлять к нему сдвиг цены другого масштаба и сохранять в 
    #    дикшенери, то такой дикшенери будет быстро расти в размере. например, если у нас 50 ячеек в каждом масштабе, а 
    #    масштабов 5 штук, то будет 50^5 ячеек = 312500000 ячеек. второй вариант - сразу посчитать разброс прогнозов и
    #    сделать для него сетку. но сетку видимо придется делать с большим количеством ячеек, иначе округления прогнозов
    #    на каждом слиянии будут давать слишком большие сдвиги.
    # мне пока больше нравится первый вариант, т.к. в нем не будет округлений вообще.
    # я подумал еще - и не уверен, что первый вариант ок. с памятью там взрыва не будет, но может быть взрыв в вычислительной
    # сложности.
    for t in range(start_idx, min(end_idx, N - tau)):
        for i in range(len(caches)):
            cache = caches[i]
            x, y, z = xs[i], ys[i], zs[i]
            cell_now = to_grid(x[t], y[t], z[t])
            if cell_now in cache:
                for future_x, count in cache[cell_now].items():
                    # Обратное преобразование: переводим future_x обратно в значение x
                    future_price_diff = x_min + (future_x / (grid_size['x'] - 1)) * (x_max - x_min)
                    # Грубая гипотеза для прогноза: будущая цена = future_price_diff * smoothed_long[t] + df['close'].iloc[t]
                    future_price = future_price_diff * smoothed_long[t] + df['close'].iloc[t]
                    current_time = df.index[t]
                    future_points.append((current_time, future_price, future_price_diff * smoothed_long[t]))
                    future_counts.append(count)

    fig, (ax, ax2) = plt.subplots(
        2,  # Два графика (2 строки)
        1,  # Один столбец
        figsize=(14, 7),  # Общий размер
        gridspec_kw={'height_ratios': [5, 2]}
    )
    ax.plot(df.index[start_idx:end_idx], df['close'].iloc[start_idx:end_idx], label="Цена BTC/USD", color='black', alpha=0.7)
    ax.plot(df.index[start_idx:end_idx], df['close'].shift(-tau).iloc[start_idx:end_idx], label="Цена (tau вперед)", color='blue', alpha=0.7)
    ax.plot(df.index[start_idx:end_idx], smoothed_short[start_idx:end_idx], label="short", color='red', alpha=0.7)
    ax.plot(df.index[start_idx:end_idx], smoothed_long[start_idx:end_idx], label="long", color='green', alpha=0.7)
    ax2.plot(df.index[start_idx:end_idx], (smoothed_short - smoothed_long)[start_idx:end_idx], label="short-long", color='black', alpha=0.7)
    ax2.plot(df.index[start_idx:end_idx], (smoothed_short - smoothed_long)[start_idx+tau:end_idx+tau], label="short-long", color='red', alpha=0.7)
    if future_points:
        future_times, future_prices, future_diffs = zip(*future_points)
        from matplotlib.colors import LinearSegmentedColormap
        # Создаём одноцветный красный градиент от светлого к насыщенному
        red_gradient = LinearSegmentedColormap.from_list("red_gradient", ["#FFEEEE", "#FF0000"])
        ax.scatter(future_times, future_prices, c=future_counts, cmap=red_gradient, alpha=0.5, s=19)
        ax2.scatter(future_times, future_diffs, c=future_counts, cmap=red_gradient, alpha=0.5, s=19)
    plt.tight_layout()
    plt.show()

# Пример: разделим данные на train/test (80/20)
N = len(df)
train_size = int(0.8 * N)
train_df = df.iloc[:train_size]
test_df  = df.iloc[train_size:]

grid_size = {'x': 50, 'y': 20, 'z': 10}
tau = 10

caches = []
bounds = []
windows = []

window_short = 31
window_long  = 61
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
caches.append(cache_3d)
bounds.append(bounds_3d)
windows.append((window_short, window_long))

window_short = 61
window_long  = 121
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
caches.append(cache_3d)
bounds.append(bounds_3d)
windows.append((window_short, window_long))

start_idx=window_long
end_idx=window_long + 300
plot_price_forecast_with_heatmap_3d(test_df, caches, bounds, windows, tau, start_idx, end_idx, grid_size)

In [ ]:
window_short = 61
window_long  = 121
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
start_idx=window_long
end_idx=window_long + 300
plot_price_forecast_with_heatmap_3d(test_df, cache_3d, bounds_3d, window_short, window_long, tau, start_idx, end_idx, grid_size)


In [ ]:
window_short = 121
window_long  = 241
print(f"tau={tau}")
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
start_idx=window_long
end_idx=window_long + 300
plot_price_forecast_with_heatmap_3d(test_df, cache_3d, bounds_3d, window_short, window_long, tau, start_idx, end_idx, grid_size)

In [ ]:
window_short = 241
window_long  = 481
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
start_idx=window_long
end_idx=window_long + 1300
plot_price_forecast_with_heatmap_3d(test_df, cache_3d, bounds_3d, window_short, window_long, tau, start_idx, end_idx, grid_size)

In [ ]:
window_short = 481
window_long  = 480*2+1
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)
start_idx=window_long
end_idx=window_long + 3000
plot_price_forecast_with_heatmap_3d(test_df, cache_3d, bounds_3d, window_short, window_long, tau, start_idx, end_idx, grid_size)

In [ ]:
tau = 150
window_short = 481
window_long  = 480*2+1
cache_3d, bounds_3d = build_phase_space_grid_3d(train_df, window_short, window_long, tau, grid_size)

In [ ]:
shift = window_long + 2500
start_idx=shift
end_idx=shift+3000
plot_price_forecast_with_heatmap_3d(test_df, cache_3d, bounds_3d, window_short, window_long, tau, start_idx, end_idx, grid_size)